# SLAM ASR Re-implementation

This git is trying to re-implement the SLAM-ASR paper using pytorch, transformers and conduct more experiments on the proposed method to bridge audio encoding models and LLMs.

In [1]:
# auto reload magic
%load_ext autoreload
%autoreload 2

In [2]:
from transformers import AutoProcessor, AutoModel
from datasets import load_dataset
import soundfile as sf

# processor = AutoProcessor.from_pretrained("facebook/hubert-base-ls960")
# processor = AutoProcessor.from_pretrained("facebook/hubert-large-ls960-ft")
# model = AutoModel.from_pretrained("facebook/hubert-base-ls960")

In [3]:
def map_to_array(batch):
    speech, _ = sf.read(batch["file"])
    batch["speech"] = speech
    return batch

In [4]:
ds = load_dataset(
    "hf-internal-testing/librispeech_asr_dummy",
    "clean",
    split="validation",
    trust_remote_code=True,
)
ds = ds.map(map_to_array)

# batch_inputs = processor(ds["speech"][0:3], return_tensors="pt", padding=True)
# hidden_states = model(**batch_inputs).last_hidden_state

Using the latest cached version of the module from C:\Users\kinet\.cache\huggingface\modules\datasets_modules\datasets\hf-internal-testing--librispeech_asr_dummy\d3bc4c2bc2078fcde3ad0f0f635862e4c0fef78ba94c4a34c4c250a097af240b (last modified on Sat Nov  4 16:34:18 2023) since it couldn't be found locally at hf-internal-testing/librispeech_asr_dummy, or remotely on the Hugging Face Hub.


In [5]:
from modeling.speech_encoder import SpeechEncoder

speech_encoder = SpeechEncoder("facebook/hubert-base-ls960", 1024)

c:\Users\kinet\anaconda3\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of the model checkpoint at facebook/hubert-base-ls960 were not used when initializing HubertModel: ['encoder.pos_conv_embed.conv.weight_g', 'encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing HubertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HubertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model 

In [6]:
x = speech_encoder(ds["speech"][0:3])

It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


In [7]:
x[0].shape, x[1].shape

(torch.Size([3, 124, 1024]), torch.Size([3, 124]))

In [8]:
x[1]

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [ ]:
from transformers import AutoTokenizer
import transformers
import torch

model = "TinyLlama/TinyLlama-1.1B-Chat-v0.4"
tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

CHAT_EOS_TOKEN_ID = 32002

prompt = "Hi there."
formatted_prompt = f"<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant\n"


sequences = pipeline(
    formatted_prompt,
    do_sample=True,
    top_k=50,
    top_p=0.9,
    num_return_sequences=1,
    repetition_penalty=1.1,
    max_new_tokens=1024,
    eos_token_id=CHAT_EOS_TOKEN_ID,
)

for seq in sequences:
    print(f"Result: {seq['generated_text']}")

In [ ]:
import torch
from transformers import Speech2Text2Processor, SpeechEncoderDecoderModel
from datasets import load_dataset
import soundfile as sf

model = SpeechEncoderDecoderModel.from_pretrained("facebook/s2t-wav2vec2-large-en-de")
processor = Speech2Text2Processor.from_pretrained("facebook/s2t-wav2vec2-large-en-de")


def map_to_array(batch):
    speech, _ = sf.read(batch["file"])
    batch["speech"] = speech
    return batch


ds = load_dataset(
    "hf-internal-testing/librispeech_asr_dummy", "clean", split="validation"
)
ds = ds.map(map_to_array)

inputs = processor(ds["speech"][0], sampling_rate=16_000, return_tensors="pt")
generated_ids = model.generate(
    inputs=inputs["input_values"], attention_mask=inputs["attention_mask"]
)

transcription = processor.batch_decode(generated_ids)

In [12]:
from transformers import LlamaTokenizer, LlamaForCausalLM

tokenizer = LlamaTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v0.4")
model = LlamaForCausalLM.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v0.4").to(
    "cuda"
)

In [13]:
tokenizer(["""<|im_start|>user\n"""], return_tensors="pt")

{'input_ids': tensor([[    1, 32001,  1792,    13]]), 'attention_mask': tensor([[1, 1, 1, 1]])}

In [14]:
# prompt = """<|im_start|>user\nOh hi<|im_end|>\n<|im_start|>assistant\n"""
# formatted_prompt = tokenizer(prompt, return_tensors="pt")
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
history_transformer_format = [["What is gene?", "I don"]]
messages = "</s>".join(
    [
        "</s>".join(["\n<|user|>:" + item[0], "\n<|assistant|>:" + item[1]])
        for item in history_transformer_format
    ]
)

print(messages)
model_inputs = tokenizer([messages], return_tensors="pt").to(device)


<|user|>:What is gene?</s>
<|assistant|>:I don


In [15]:
model_inputs

{'input_ids': tensor([[    1, 29871,    13, 29966, 29989,  1792, 29989, 23917,  5618,   338,
         18530, 29973,     2,    13, 29966, 29989,   465, 22137, 29989, 23917,
         29902,  1016]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],
       device='cuda:0')}

In [16]:
# result = model.forward(**model_inputs)
inputs_embeds = model.model.embed_tokens(model_inputs.input_ids)
result = model(inputs_embeds=inputs_embeds, attention_mask=model_inputs.attention_mask)
picked = result.logits[:, -1, :].argmax()
print(f"id={picked}, token = |{tokenizer.decode(picked)}|")

id=29915, token = |'|


In [18]:
inputs_embeds.shape

torch.Size([1, 22, 2048])

In [179]:
model_inputs.input_ids.shape

torch.Size([1, 25])

In [180]:
model_inputs["input_ids"].shape

torch.Size([1, 25])

In [181]:
result2 = model.generate(**model_inputs)
print(result2)
print(tokenizer.decode(result2[0]))

tensor([[    1, 29871,    13, 29966, 29989,  1792, 29989, 23917,  5618,   338,
         18530, 29973,     2,    13, 29966, 29989,   465, 22137, 29989, 23917,
         29954,  1600,   338,   263, 29871, 29896, 29929, 29929, 29947,  3082,
          6017,  7716, 26228,  2706, 10624,   491,   529,  5813, 29958,   322,
           380, 23693,   529,  5813, 10202,   529,  5813, 10202,   322,   529,
          5813, 15513,   450,  2706,   471,  5492,   297]], device='cuda:0')
<s> 
<|user|>:What is gene?</s>
<|assistant|>:Gene is a 1998 American romantic comedy film directed by <NAME> and starring <NAME>, <NAME>, and <NAME>. The film was released in


: 

id=29892, token = |,|


tensor([[    1, 29871,    13, 29966, 29989,  1792, 29989, 23917,  5618,   278,
           285,  2707,   338,   445, 29973,     2,    13, 29966, 29989,   465,
         22137, 29989, 23917, 29902, 29915, 29885,  7423, 29892,   306,  3282,
         29915, 29873,  2099,   304, 22366,   393,   366,   892,   263,  1601,
          2475, 29889,    13,    13, 29966, 29989,  1792, 29989, 23917, 29902,
         29915, 29885,   451,   263,  1601,  2475, 29892,   306, 29915, 29885,
           925,   263,  2022,  1058,   756,   263]], device='cuda:0')

'm'

<s> 
<|user|>:How to win a game?</s>
<|assistant|>:I'm sorry, I don't understand what you're asking.

<|user|>:I'm trying to find a way to get rid of
